In [ ]:
import pandapower as pp
import pandapower.networks
import pandas as pd
import numpy as np
import tensorflow as tf
import os 

import matplotlib.pyplot as plt
%matplotlib inline

from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl

In [ ]:
net = pp.networks.mv_oberrhein()
net1, net2 = pp.networks.mv_oberrhein(separation_by_sub=True)

In [ ]:
net = net1

In [ ]:
#Plot network and loadings
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import vlevel_plotly
from pandapower.plotting.plotly import pf_res_plotly

# fig = simple_plotly(net2, bus_size=5, ext_grid_size=10)
# _ = vlevel_plotly(net, bus_size=5, ext_grid_size=10)
# fig = pf_res_plotly(net, bus_size=5, ext_grid_size=10)
# fig.write_html(f"images/MVOberrhein/Half2.html")
# fig.write_image(f"images/MVOberrhein/Half2.png")

In [ ]:
def num_foreach_element(arr):
	d = {}
	for i in arr.to_numpy():
		i = i[0]
		if(i in d.keys()):
			d[i]+=1
		else:
			d[i] = 1
	return d

In [ ]:
input_dir = './TimeSeries/1-MVLV-rural-all-0-sw/'
n_timesteps = 4 * 24 * 365

#Loads dataset
profile_load = pd.DataFrame()
n_load = len(net.load)
n_res = len(net.sgen)
# The parameter “sR” generally describes the nominal apparent power of power plants, distributed energy resources and loads
loads = pd.read_csv(f'{input_dir}Load.csv', sep=';')
loads_timeseries = pd.read_csv(f'{input_dir}LoadProfile.csv', sep=';')

RESs = pd.read_csv(f'{input_dir}RES.csv', sep=';')
RESs_timeseries = pd.read_csv(f'{input_dir}RESProfile.csv', sep=';')
'''
# Papers:
# -SimBench—A Benchmark Dataset of Electric Power Systems to Compare Innovative Solutions Based on Power Flow Analysis
# -https://publica.fraunhofer.de/eprints/urn_nbn_de_0011-n-5554297.pdf
Commercial enterprises (G), households (H), agricultural holdings (L) and industrial companies
(BL/BW) were considered as accumulated consumers, while the provided time series for electric
vehicles (EVs) and heat pumps (HPs) were interpreted as individual consumers.
#print(set(loads_timeseries.columns)) 
'''
loads_type = loads.loc[:n_load-1,['profile']] #loc ranges is [0,n], not [0,n[. Find a better/more uniform way to select the loads
temp_profile_p = []
temp_profile_q = []
print(f'Load elements by type: {num_foreach_element(loads_type)}')
for l in loads_type.to_numpy():
	temp_profile_p.append(loads_timeseries[f'{l[0]}_pload'])
	temp_profile_q.append(loads_timeseries[f'{l[0]}_qload'])

#Loads p (in MW) 
profile_load_p = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_load_p.columns = range(0,n_load)
ds = DFData(profile_load_p)
cc = ConstControl(net, 'load', 'p_mw', element_index=net.load.index, profile_name=profile_load_p.columns, data_source=ds, drop_same_existing_ctrl=True)

#Loads q (in MVar)
profile_load_q = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_load_q.columns = range(0,n_load)
ds = DFData(profile_load_q)
cc = ConstControl(net, 'load', 'q_mvar', element_index=net.load.index, profile_name=profile_load_q.columns, data_source=ds, drop_same_existing_ctrl=True)

#RES p (in MW)
res_to_add = pd.DataFrame(['WP4','WP7','WP4','WP7','WP4','WP7'], columns=['profile'])
RESs_type = RESs.loc[:n_res-1-len(res_to_add),['profile']]
RESs_type = pd.concat([RESs_type, res_to_add])
temp_profile_p = []
print(f'RES elements by type: {num_foreach_element(RESs_type)}')
for l in RESs_type.to_numpy():
	temp_profile_p.append(RESs_timeseries[f'{l[0]}'])
	# temp_profile_p.append(RESs_timeseries[f'{l[0]}_pload']) #Q values are not required

#RESs p
profile_res_p = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_res_p.columns = range(0,n_res)
ds = DFData(profile_res_p)
cc = ConstControl(net, 'sgen', 'p_mw', element_index=net.sgen.index, profile_name=profile_res_p.columns, data_source=ds, drop_same_existing_ctrl=True)

In [ ]:
time_steps = range(0,n_timesteps)

output_dir = os.path.join(input_dir,'Results')
ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".xlsx", log_variables=list())

#Save time series (output)
# these variables are saved to the harddisk after / during the time series loop
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_line', 'i_ka')
#Add net.res_trafo['loading_percent']

print('Time steps: ',len(time_steps), '. Num Loads: ', net.load.index.shape, '. Load p and q: ', profile_load_p.shape, profile_load_q.shape, '. Num RESs: ', net.sgen.index.shape, '. RESs p: ',profile_res_p.shape)
run_timeseries(net,time_steps)

#Save time series (input)
path = os.path.join(output_dir, "loads_p.xlsx")
profile_load_p.to_excel(path)
path = os.path.join(output_dir, "loads_q.xlsx")
profile_load_q.to_excel(path)
path = os.path.join(output_dir, "RESs_p.xlsx")
profile_res_p.to_excel(path)